
# Note book for the project 2 

Kaggle competition link: [Submition]('https://www.kaggle.com/c/epfml17-text/submit')

## Pipeline: 


### Create cooc matrix

1. sh build_vocab.sh
2. sh cut_vocab.sh
3. python3 pickle_vocab.py
4. python3 cooc.py

Now given the co-occurrence matrix and the vocabulary, it is not hard to train GloVe word embeddings, that is to compute an embedding vector for wach word in the vocabulary. We suggest to implement SGD updates to train the matrix factorization, as in

5. python3 glove_template.py

Once you tested your system on the small set of 10% of all tweets, we suggest you run on the full datasets pos_train_full.txt, neg_train_full.txt

### Building a Text Classifier:

1. Construct Features for the Training Texts: Load the training tweets and the built GloVe word embeddings. Using the word embeddings, construct a feature representation of each training tweet (by averaging the word vectors over all words of the tweet).

2. Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels indicate if a tweet used to contain a 🙂 or 🙁 smiley.

3. Prediction: Predict labels for all tweets in the test set.

4. Submission / Evaluation: Submit your predictions to kaggle, and verify the obtained misclassification error score. (You can also use a local separate validation set to get faster feedback on the accuracy of your system). Try to tune your system for best evaluation score.

### Extensions:
Naturally, there are many ways to improve your solution, both in terms of accuracy and computation speed. More advanced techniques can be found in the recent literature.


TODO :

- implement cross-validation
- 

Importing usefull library

In [24]:
import numpy as np
import pandas as pd
%matplotlib inline
#!/usr/bin/env python3
from scipy.sparse import *
from sklearn import linear_model, preprocessing
import numpy as np
import pickle
import random

# 0.Variables


In [67]:
file_storing_ts = 'embeddings.npy'
file_storing_pos_ts_tweets = 'train_pos.txt'
file_storing_neg_ts_tweets = 'train_neg.txt'
file_storing_te_tweets = 'test_data.txt'
file_storing_relevant_vocab = 'new_vocab'
pertinence = 200
nb_dim = 20



In [9]:
def word_embeddings(file_name='cooc_full.pkl', destination = file_storing_ts):
    print("loading cooccurrence matrix")
    with open(file_name, 'rb') as f:
        cooc = pickle.load(f)
    print("{} nonzero entries".format(cooc.nnz))
    
    nmax = 100
    print("using nmax =", nmax, ", cooc.max() =", cooc.max())
    print("initializing embeddings")
    embedding_dim = 20
    xs = np.random.normal(size=(cooc.shape[0], embedding_dim))
    ys = np.random.normal(size=(cooc.shape[1], embedding_dim))
    eta = 0.001
    alpha = 3 / 4
    epochs = 3
    
    for epoch in range(epochs):
        print("epoch {}".format(epoch))
        for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
            
            f = ((n / nmax)**alpha) if n < nmax else 1
            inter_cost = (xs[ix]@(ys[jy]) - np.log(n))
            # We compute the gradients for both context and main vector words
            grad_main = f * inter_cost * ys[jy]
            grad_context = f * inter_cost * xs[ix]
    
            # Update the vector words
            xs[ix] = xs[ix] - (eta * grad_main)
            ys[jy] = ys[jy] - (eta * grad_context)
            
    np.save(destination, xs)
 


# 1.Construct words_embeddings for training set


In [10]:
word_embeddings()


loading cooccurrence matrix
46597044 nonzero entries
using nmax = 100 , cooc.max() = 2599902
initializing embeddings
epoch 0
epoch 1
epoch 2


# 2.Load words for training set


In [90]:
with open('vocab.pkl', 'rb') as f :
    vocab = pickle.load(f)
xs = np.load(file_storing_ts)


# 3.Construct features for training tweets


In [43]:
new_vocab = pd.read_csv(filepath_or_buffer="./new_vocab", sep=" ")
weights = new_vocab[["word", "ratio"]]
weights = dict(zip(weights.word, weights.ratio))


In [82]:
def construct_features(tweets):
    features = []
    invalid_features = 0;

    for line in tweets:
        #we differentiate tweets containing pertinent words, those in 'weights'
        sum_w_pertinent = np.zeros(nb_dim)
        sum_w_others = np.zeros(nb_dim)
        
        count_pertinent = 0
        count_other = 0
        
        for word in line.split():
            local_w = vocab.get(word, -1)
            if local_w != -1:
                weight = weights.get(word, -1)
                if weight != -1:                
                    
                    #If the word is pertinent, we add its word representation to others pertinents word's representation
                    count_pertinent += 1 + (weight*pertinence)
                    sum_w_pertinent += xs[local_w] * (1 + (weight*pertinence))
                    
                else:
                    
                    #If the word is not pertinent, we add its representation to non-pertinent words representations
                    count_other += 1 
                    sum_w_others += xs[local_w] 
     
            # If we found pertinent words, we only use them
        if(count_pertinent != 0):            
            features.append(sum_w_pertinent/count_pertinent)
            
            #if we found only non-pertinent words, we use them anyway
        elif count_other!= 0:
            features.append(sum_w_others/count_other)
            
            #if we did not find words that have representation, we do not try to create features and signal how many they were
        else:
            invalid_features += 1
            
    features = np.array(features)
    
    return features, invalid_features



In [83]:
pos_tweets = open(file_storing_pos_ts_tweets, 'r').readlines() 
neg_tweets = open(file_storing_neg_ts_tweets, 'r').readlines()

pos_features, invalid_pos = construct_features(pos_tweets)
neg_features, invalid_neg = construct_features(neg_tweets)


# Train the linear classifier


In [49]:
type(pos_tweets)

list

In [87]:
clf = linear_model.SGDClassifier(n_iter=400, warm_start=True);
X = np.concatenate((pos_features, neg_features))
scaler = preprocessing.StandardScaler()
Y = np.concatenate((np.ones(len(pos_features)), np.full(len(neg_features), -1)))
X = scaler.fit_transform(X, Y)
clf = clf.fit(X, Y)

                   

In [88]:
prediction = clf.predict(X)


In [89]:
print("accuracy : ", (1 - (np.sum(np.abs(Y-prediction))/(2*len(Y)))) * 100, "%")

accuracy :  63.2153446905 %


In [ ]:
Y.shape

In [ ]:
np.save?